In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
#import pyperclip
from collections import OrderedDict as OD
#from pint import UnitRegistry

from d3py import *
from CEB import *
from plotcurves import *
from CapModel import *
#from transformation import *

: 

# DIF

In [2]:
f_c=43.9

DIFCompressionCurve={
    'array'     : DIF_c(f_c),
    'lable'     : 'CEB-FIP Compression',
    'color'     : 'red',
    'linestyle' : '-',
    'linewidth' : 2.0  
}

DIFTensionCurve={
    'array'     : DIF_t(f_c),
    'lable'     : 'CEB-FIP Tension',
    'color'     : 'blue',
    'linestyle' : '-',
    'linewidth' : 2.0  
}

DIFCSCMCompressionCurve={
    'array'     : DIFCSCM_c(f_c, rev=1),
    'lable'     : 'CSCM Compression',
    'color'     : 'red',
    'linestyle' : ':',
    'linewidth' : 2.0  
}

DIFCSCMTensionCurve={
    'array'     : DIFCSCM_t(f_c, rev=1),
    'lable'     : 'CSCM Tension',
    'color'     : 'blue',
    'linestyle' : ':',
    'linewidth' : 2.0  
}

DIFRMTensionCurve={
    'array'     : np.genfromtxt('./data/Malvar & Ross [1998] tension.txt', skip_header =1).T,
    'lable'     : 'Malvar & Ross [1998] tension',
    'color'     : 'red',
    'linestyle' : '--',
    'linewidth' : 2.0  
}
DIFRMCompressionCurve={
    'array'     : np.genfromtxt('./data/Malvar & Ross [1998] compression.txt', skip_header =1).T,
    'lable'     : 'Malvar & Ross [1998] compression',
    'color'     : 'blue',
    'linestyle' : '--',
    'linewidth' : 2.0  
}

In [ ]:
plot(
    plotStile(
        xLabel = 'Strain Rate, 1/sec',
        yLabel = 'DIF',
        xScale = 'log',
        xLimBottom = 1E-3,
        xLimTop = 1e3,
        yLimBottom = 0,
        yLimTop = 6
    ), 
    DIFCompressionCurve,DIFTensionCurve, DIFCSCMCompressionCurve, DIFCSCMTensionCurve, DIFRMTensionCurve, DIFRMCompressionCurve
)

#

# Stress-strain curve

In [4]:
f_c=43.9

tensionCurve = dict()
tensionCurve['array']     = CEB(f_c)['tension_curve']
tensionCurve['lable']     = 'Unconfind tension'
tensionCurve['color']     = 'red'
tensionCurve['linestyle'] ='-'
tensionCurve['linewidth'] = 2.0

compressionCurve = dict()
compressionCurve['array']     = CEB(f_c)['compression_curve']
compressionCurve['lable']     = 'Unconfiend compression'
compressionCurve['color']     = 'blue'
compressionCurve['linestyle'] ='-'
compressionCurve['linewidth'] = 2.0

crackOpeningCurve = dict()
crackOpeningCurve['array']     = CEB(f_c)['crack_opening_curve']
crackOpeningCurve['lable']     = 'Crack opening'
crackOpeningCurve['color']     = 'green'
crackOpeningCurve['linestyle'] ='-'
crackOpeningCurve['linewidth'] = 2.0

In [ ]:
plot(
    plotStile(
        xLabel = 'Strain',
        yLabel = 'Stress, MPa'
    ), 
    tensionCurve
)

In [ ]:
plot(
    plotStile(
        xLabel = 'Crack opening, mm',
        yLabel = 'Stress, MPa'
    ), 
    crackOpeningCurve
)

In [ ]:
plot(
    plotStile(
        xLabel = 'Strain',
        yLabel = 'Stress, MPa'
    ), 
    compressionCurve
)

# Plot and check

## Smooth transition between the tensile and compressive pressure regions
This is accomplished by setting:
$$\alpha_{1} – \lambda_{1} = 0.5774$$ 

$$Q_{1}\Bigr|_{P=0} = 0.5774$$ 

In [ ]:
f_c=40
J=0 

print("alpha_1(rev=1) = {0}".format(alpha_1(f_c,rev=1)))
print("lamda_1(rev=1) = {0}".format(lamda_1(f_c,rev=1)))
print("alpha_1(rev=1)-lamda_1(rev=1) = {0}".format(alpha_1(f_c,rev=1)-lamda_1(f_c,rev=1)))

print("alpha_1(rev=3) = {0}".format(alpha_1(f_c,rev=3)))
print("lamda_1(rev=3) = {0}".format(lamda_1(f_c,rev=3)))
print("alpha_1(rev=3)-lamda_1(rev=3) = {0}".format(alpha_1(f_c,rev=3)-lamda_1(f_c,rev=3)))

print("Q1(P=0, rev=1) = {0}".format(Q_1(f_c,J,1)))
print("Q1(P=0, rev=3) = {0}".format(Q_1(f_c,J,3)))

$$\alpha_{2} – \lambda_{2} = 0.5$$

$$Q_{2}\Bigr|_{P=0} = 0.5$$

In [ ]:
print("alpha_2(rev=1)-lamda_2(rev=1) = {0}".format(alpha_2(f_c,rev=1)-lamda_2(f_c,rev=1)))
print("alpha_2(rev=3)-lamda_2(rev=3) = {0}".format(alpha_2(f_c,rev=2)-lamda_2(f_c,rev=2)))

print("Q_2(P=0, rev=1) = {0}".format(Q_2(40,0,1)))
print("Q_2(P=0, rev=2) = {0}".format(Q_2(40,0,2)))

In [10]:
#Q1 Mhor-Coulomb fit 
def Q1MC(f_c, I, rev=1):
    return np.sqrt(3)*Q_2(f_c,I,rev)/(1+Q_2(f_c, I, rev))

In [11]:
#Q2 Mhor-Coulomb fit 
def Q2MC(f_c, I, rev=1):
    return TXE(f_c, I, rev)/TXC(f_c, I, rev)

## Willam-Warnke fit
$$
Q_{1}=\frac{\sqrt{3}(1-Q^2_{2})+(2Q_{2}-1)\sqrt{3(1-Q^2_{2})+5Q^2_{2}-4Q_{2}}}{3(1-Q^2_{2})+(1-2Q_{2})^2}
$$

In [12]:
#Q2 Willam-Warnke fit
def Q1WW(f_c, I, rev=1):
    q=(1-pow(Q_2(f_c, I, rev),2))
    return (np.sqrt(3)*q+(2*Q_2(f_c, I, rev)-1)*np.sqrt((3*q)+5*pow(Q_2(f_c, I, rev),2)-4*Q_2(f_c, I, rev)))/(3*q+pow(1-2*Q_2(f_c, I, rev),2))

In [13]:
def TORMC(f_c, J, rev=1):
    return Q1MC(f_c,J,rev)*TXC(f_c,J,rev)

In [14]:
def TXEMC(f_c, J, rev=1):
    return Q2MC(f_c,J,rev)*TXC(f_c,J,rev)

In [15]:
def TORWW(f_c, J, rev=1):
    return Q1WW(f_c,J,rev)*TXC(f_c,J,rev)

In [ ]:
f_c = 40 
X = np.linspace(0, 1200, 100) 

plotTwoCurves(
    x      = X,
    y1     = hydrostaticCompressionParameters(f_c,X,rev=1),
    y2     = hydrostaticCompressionParameters(f_c,X,rev=2),
    title  = 'Hydrostatic Compression Parameters',
    label1 = 'rev 1', 
    label2 = 'rev 2', 
    xLabel = '$J_{1}$, MPa',
    yLabel = '$-ln(\epsilon^0_{V}/W)$')

In [17]:
f_c = 40 
TXC_1 = dict()
TXC_2 = dict()
TXC_3 = dict()

TOR_1 = dict()
TOR_2 = dict()
TOR_3 = dict()

TXE_1 = dict()
TXE_2 = dict()
TXE_3 = dict()

J = np.linspace(-3*CEB(f_c, d_max=10)['f_tt'], X0(f_c, rev=1), 100) 

TXC_1['array'] = np.vstack((J,TXC(f_c,J,rev=1)))
TXC_1['lable'] = '$TXC_{1}$'
TXC_1['color'] = 'red'
TXC_1['linestyle'] ='-'
TXC_1['linewidth'] = 2.0

TXC_2['array'] = np.vstack((J,TXC(f_c,J,rev=2)))
TXC_2['lable'] = '$TXC_{2}$'
TXC_2['color'] = 'red'
TXC_2['linestyle'] =':'
TXC_2['linewidth'] = 2.0

TXC_3['array'] = np.vstack((J,TXC(f_c,J,rev=3)))
TXC_3['lable'] = '$TXC_{3}$'
TXC_3['color'] = 'red'
TXC_3['linestyle'] ='--'
TXC_3['linewidth'] = 2.0

TOR_1['array'] = np.vstack((J,TOR(f_c,J,rev=1)))
TOR_1['lable'] = '$TOR_{1}$'
TOR_1['color'] = 'blue'
TOR_1['linestyle'] ='-'
TOR_1['linewidth'] = 2.0

TOR_2['array'] = np.vstack((J,TOR(f_c,J,rev=2)))
TOR_2['lable'] = '$TOR_{2}$'
TOR_2['color'] = 'blue'
TOR_2['linestyle'] =':'
TOR_2['linewidth'] = 2.0

TOR_3['array'] = np.vstack((J,TOR(f_c,J,rev=3)))
TOR_3['lable'] = '$TOR_{3}$'
TOR_3['color'] = 'blue'
TOR_3['linestyle'] ='--'
TOR_3['linewidth'] = 2.0

TXE_1['array'] = np.vstack((J,TXE(f_c,J,rev=1)))
TXE_1['lable'] = '$TXE_{1}$'
TXE_1['color'] = 'green'
TXE_1['linestyle'] ='-'
TXE_1['linewidth'] = 2.0

TXE_2['array'] = np.vstack((J,TXE(f_c,J,rev=2)))
TXE_2['lable'] = '$TXE_{2}$'
TXE_2['color'] = 'green'
TXE_2['linestyle'] =':'
TXE_2['linewidth'] = 2.0

TXE_3['array'] = np.vstack((J,TXE(f_c,J,rev=3)))
TXE_3['lable'] = '$TXE_{3}$'
TXE_3['color'] = 'green'
TXE_3['linestyle'] ='--'
TXE_3['linewidth'] = 2.0

In [ ]:
plot(
    plotStile(
        xLabel = '$I_{1}$, MPa',
        yLabel = '$\sqrt{J_{2}}$, MPa'
    ), 
    TXC_1,
    TXC_2,
    TXC_3
)

In [ ]:
plot(
    plotStile(
        xLabel = '$J_{1}$, MPa',
        yLabel = '$\sqrt{J_{2}}$, MPa'
    ), 
    TOR_1,
    TOR_2,
    TOR_3
)

In [ ]:
plot(
    plotStile(
        xLabel = '$I_{1}$, MPa',
        yLabel = '$\sqrt{J_{2}}$, MPa'
    ), 
    TXE_1,
    TXE_2,
    TXE_3
)

In [ ]:
plot(
    plotStile(
        xLabel = '$I_{1}$, MPa',
        yLabel = '$\sqrt{J_{2}}$, MPa'
    ), 
    TXC_1,
    TXE_1,
    TOR_1
)

In [ ]:
plot(
    plotStile(
        xLabel = '$I_{1}$, MPa',
        yLabel = '$\sqrt{J_{2}}$, MPa'
    ), 
    TXC_2,
    TXE_2,
    TOR_2
)

In [ ]:
plot(
    plotStile(
        xLabel = '$I_{1}$, MPa',
        yLabel = '$\sqrt{J_{2}}$, MPa'
    ), 
    TXC_3,
    TXE_3,
    TOR_3
)

# Keyword output

In [ ]:
f_c = 33


d_max = 18
rho = 2.4E-09

#rho = 0.0024

MAT_CSCM = CSCM(
    f_c   = f_c,
    dmax  = d_max,
    irate = 'on',
    erode = 'off', 
    pred  = 'off', 
    recov = 'full'
    )
print(keyword2text(MAT_CSCM) + CEBout(f_c, d_max, rho ))